## Loading all the required Packages

In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.models import *
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from tensorflow.keras.layers import Dropout
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from keras.utils.vis_utils import plot_model

%matplotlib inline

## Loading the Datasets

#### We will using the ImageDataGenerator from keras package to load all our image files directly from the directory on our local machine. 

#### The first block contains the path files where your dataset folder should be located. Here, in the example the mask folder contains the sub folder train, test and val. Inside each sub-folder there are two more folders called "Mask" and "No Mask". Each folder has a certain number of image files(.jpg)

#### The ImageDataGenerator function helps us to directly import the image files(.jpg) from our directory.


In [ ]:
'''
Example code:

train_path = 'C:\\mask\\train'
test_path  = 'C:\\mask\\test'
val_path   = 'C:\\mask\\val'

'''

train_path = 'C:\\mask\\train'
test_path  = 'C:\\mask\\test'
val_path   = 'C:\\mask\\val'

### The following lines of code load our image files(.jpg) into three different tensor DataIterator namely:
#### 1. train_batches = containing all the train images and their labels.
#### 2. val_batches = containing all the validation images and their labels.
#### 2. train_batches = containing all the test images and their labels.

In [ ]:
train_batches  = ImageDataGenerator(preprocessing_function = keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path, target_size=(224,224), classes=['0 (Mask Off)', '1 (Mask On)'], shuffle = True, batch_size=10)

val_batches  = ImageDataGenerator(preprocessing_function = keras.applications.mobilenet.preprocess_input).flow_from_directory(
    val_path, target_size=(224,224), classes=['0 (Mask Off)', '1 (Mask On)'], shuffle = True, batch_size=10)

test_batches  = ImageDataGenerator(preprocessing_function = keras.applications.mobilenet.preprocess_input).flow_from_directory(
    test_path, target_size=(224,224), classes=['0 (Mask Off)', '1 (Mask On)'], shuffle = True, batch_size=194)

In [ ]:
x_test, y_test = next(test_batches) #This helps us to extract all the test image files and their labels 
                                    #into two different varibale

In [ ]:
print("Number of Test Examples:", x_test.shape[0])
print("Number of unique values of Y:", y_test.shape[1])
print("Size of Each image:", x_test.shape[1], "*", x_test.shape[2], "* 1")

## Loading the pre-trained MobileNetV2 CNN Architecture

In [ ]:
model = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
model.summary() #Prints the model summary


#### Plots a Visual Map of the Neural Network

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True)

##  


##  


### The model looks good but it needs to be tuned to be able to detect human faces and should able to differentiate between people wearning face masks and people not wearning face masks

#### The original Network contains an ReLu activation layer as out put which needs to be replaced with a softamac function of two classes for our binary classification

In [ ]:
x = model.output                               #Removes the output layer from the original Model
x = AveragePooling2D(pool_size=(7, 7))(x)      #Adds an AveragePooling Layer of size 7*7
x = Flatten(name="flatten")(x)                 #Flatten the Image
x = Dense(128, activation="relu")(x)           #Apply a ReLu activation function
x = Dropout(0.5)(x)
x = Dense(2, activation="softmax")(x)          #Apply a softmax function of 2 classes for binary classification

model = Model(inputs = model.input, outputs = x)  #Using Keras Model Object to create the Modified Neural Network

##  


### Plotting the graph of the Modified Neural Network

#### You can compare the graph of this model with the earlier plotted graph and observe the changes in the total number of layers and parameters

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True)

##  


### Setting Optimizing Paramteres

#### After multiple iterations the following optimizing parameters seem to work pretty well in helping our model achieve a higher accuracy

In [ ]:
opt = Adam(lr=1e-4, decay=1e-4 / 20)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
for layer in model.layers:
    layer.trainable = False              

##  


### Traning the model and passing the validation batches to test the train and validation accuracy

#### The moedl.fit function from Keras helps us to do this.

#### Depending on our computer specifications this might take some time. On my system it takes around 30 mins.

#### So, untill the model gets trained you can get yourself some coffee. :)

In [ ]:
model.fit(train_batches, batch_size = 32, steps_per_epoch= 18 , validation_data=(val_batches), validation_steps= 2, epochs=40, verbose=1)

##  


### It's time to evaluate our model by running it on your Test Set

#### The evaluate fucntion helps to pass the x_test containg the test images and the y_test containing the labels. The batch size is equal to the total number of images in the test set which means that the model will be run on all the images in test set at once.

In [ ]:
##  
preds = model.evaluate(x=x_test, y=y_test, batch_size=194)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

#### With an accuracy of 99.48% on your test set we can say that our model is doing very well.

#### It is very imortant to save our model once the model is trained. The saved model will contain the weights that thus eliminate the need to retrain the model again whenever we want to use it.

In [ ]:
model.save('mask_model.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
mask_model = load_model('mask_model.h5') #model = load_model('my_model')

## Our Model is doing pretty good. Let's test it on some local, new Images.

### Why not tot est it out on your own image ?
### Set the Path to your Image in img_path variable. 

In [ ]:
'''
Example Code :

img_path = 'C:/Users/My_picture.jpg'

'''

img_path = 'PATH'
img = image.load_img(img_path, target_size=(224, 224))
imshow(img)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

if (model.predict(x).argmax(axis=1)) == 0:
    print("Mask Off")
else:
    print("Mask On")

##  


## THANK YOU FOR READING !!

### If you want to know how you can implement this model to classify live video stream, do check out the other files in the repository. 